<a href="https://colab.research.google.com/github/viktoriya-bel/Tensorflow-labs/blob/main/Exercise4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ниже приведен код со ссылкой на набор данных «happy» или «sad», содержащий 80 изображений, 40 счастливых и 40 грустных.
Создайте сверточную нейронную сеть, которая обучается со 100% точностью на этих изображениях, с отменой обучения при достижении точности обучения .999

In [1]:
import tensorflow as tf
import os
import zipfile
#
!wget --no-check-certificate "https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"


--2023-05-23 16:45:33--  https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip [following]
--2023-05-23 16:45:34--  https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.07s   

2023-05-23 16:45:34 (34.8 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/26

In [2]:

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-16.png', 'happy1-09.png', 'happy2-15.png', 'happy1-12.png', 'happy1-06.png', 'happy2-00.png', 'happy2-02.png', 'happy1-04.png', 'happy1-07.png', 'happy2-14.png']
['sad2-01.png', 'sad2-00.png', 'sad1-00.png', 'sad2-18.png', 'sad1-18.png', 'sad2-19.png', 'sad2-10.png', 'sad1-19.png', 'sad1-13.png', 'sad1-03.png']


In [3]:
import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-16.png', 'happy1-09.png', 'happy2-15.png', 'happy1-12.png', 'happy1-06.png', 'happy2-00.png', 'happy2-02.png', 'happy1-04.png', 'happy1-07.png', 'happy2-14.png']
['sad2-01.png', 'sad2-00.png', 'sad1-00.png', 'sad2-18.png', 'sad1-18.png', 'sad2-19.png', 'sad2-10.png', 'sad1-19.png', 'sad1-13.png', 'sad1-03.png']


In [30]:

DESIRED_ACCURACY = 0.999

# Класс наследованный от основного класса tf.keras.callbacks.Callback
class myCallback(tf.keras.callbacks.Callback):
      # Вызывается в начале эпохи во время тренировки.
      def on_epoch_end(self, epoch, logs={}):
          if logs.get('accuracy') is not None and logs.get('accuracy') > DESIRED_ACCURACY: 
              print("\nДостигнута точность 99.9%, обучение прерывается!") 
              # Stop training once the above condition is met
              self.model.stop_training = True

callbacks = myCallback()

# Этот блок кода должен задать и скомпилировать модель. Пожалуйста, имейте в виду, что размер изображений 150 X 150

# Создаем модель, имеющую 7 слоев:
# - convolutions и max pooling
# - Flatten слой, должен принимать входные данные того же формата, что и изображения
# - Dense слой с 512 нейронами и функцией активации ReLU
# - Dense слой с 1 нейроном и функцией активации sigmoid
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001), 
              metrics=['accuracy']
    )
    

  # Этот блок кода должен создать экземпляр ImageDataGenerator с именем train_datagen
  # и train_generator, вызовом train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

# target_size  150 X 150.
train_generator = train_datagen.flow_from_directory(
        "/tmp/h-or-s",
        target_size=(150,150),
        batch_size=128,
        class_mode = 'binary')

#validation_datagen = ImageDataGenerator(rescale=1/255)
#validation_generator = validation_datagen.flow_from_directory('/tmp/h-or-s', target_size=(150, 150),batch_size=32, class_mode='binary')

# ожидаемая строка вывода: 'Found 80 images belonging to 2 classes'
# model fitting
history = model.fit(train_generator, steps_per_epoch = None, epochs = 24, verbose = 1, callbacks = [callbacks])
print(history.epoch, history.history['accuracy'][-1])
# ожидаемая строка вывода: "Достигнута точность 99.9%, обучение прерывается!"


Found 80 images belonging to 2 classes.
Epoch 1/24
1/1 [==============================] - 2s 2s/step - loss: 0.6861 - accuracy: 0.7375
Epoch 2/24
1/1 [==============================] - 1s 1s/step - loss: 1.4944 - accuracy: 0.5000
Epoch 3/24
1/1 [==============================] - 1s 1s/step - loss: 4.8993 - accuracy: 0.5000
Epoch 4/24
1/1 [==============================] - 1s 1s/step - loss: 0.6928 - accuracy: 0.5000
Epoch 5/24
1/1 [==============================] - 1s 1s/step - loss: 0.7251 - accuracy: 0.5000
Epoch 6/24
1/1 [==============================] - 1s 1s/step - loss: 0.6980 - accuracy: 0.5000
Epoch 7/24
1/1 [==============================] - ETA: 0s - loss: 0.6535 - accuracy: 1.0000
Достигнута точность 99.9%, обучение прерывается!
1/1 [==============================] - 1s 1s/step - loss: 0.6535 - accuracy: 1.0000
[0, 1, 2, 3, 4, 5, 6] 1.0
